In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.stats import norm
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML


# Black-Scholes analytical model (option price)

In [2]:


def black_scholes(S, K, T, r, sigma, option_type):
    if T <= 0:
        return 0 
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        
    return price


# Monte Carlo simulation with confidence interval

In [3]:


def monte_carlo(S, K, T, r, sigma, num_simulations, num_steps, confidence_level, option_type):
    if T <= 0:
        return 0, None, 0  
    dt = T / num_steps
    nudt = (r - 0.5 * sigma ** 2) * dt
    sidt = sigma * np.sqrt(dt)
    
    paths = np.zeros((num_simulations, num_steps + 1))
    paths[:, 0] = S
    
    for i in range(1, num_steps + 1):
        z = np.random.standard_normal(num_simulations)
        paths[:, i] = paths[:, i-1] * np.exp(nudt + sidt * z)
    
    if option_type == 'call':
        payoffs = np.maximum(paths[:, -1] - K, 0)
    else :
        payoffs = np.maximum(K - paths[:, -1], 0)
    
    option_prices = np.exp(-r * T) * payoffs
    
    mean_price = np.mean(option_prices)
    std_error = np.std(option_prices, ddof=1) / np.sqrt(num_simulations)
    
    confidence_interval = norm.ppf((1 + confidence_level) / 2) * std_error
    
    return mean_price, paths, confidence_interval


# Check put-call parity

In [4]:


def check_put_call_parity(S, K, T, r, call_price, put_price):
    lhs = call_price + K * np.exp(-r * T)
    rhs = put_price + S
    return np.isclose(lhs, rhs, rtol=1e-8), lhs, rhs


# GUI

In [5]:

def black_scholes_monte_carlo_gui():
    S_input = widgets.FloatText(value=100, description='Stock Price (S) (€):', style={'description_width': 'initial'})
    K_input = widgets.FloatText(value=100, description='Strike Price (K) (€):', style={'description_width': 'initial'})
    T_input = widgets.FloatText(value=1, description='Time to Maturity (T) (years):', style={'description_width': 'initial'})
    r_input = widgets.FloatText(value=0.03, description='Risk-free Rate (r):', style={'description_width': 'initial'})
    sigma_input = widgets.FloatText(value=0.4, description='Volatility (σ):', style={'description_width': 'initial'})
    num_simulations_input = widgets.IntText(value=1000, description='Number of Simulations:', style={'description_width': 'initial'})
    shown_simulations_input = widgets.IntText(value=100, description='Shown Simulations:', style={'description_width': 'initial'})
    num_steps_input = widgets.IntText(value=1000, description='Number of Steps:', style={'description_width': 'initial'})
    confidence_level_input = widgets.FloatSlider(value=0.95, min=0.6, max=0.99, step=0.01, description='Confidence Level:', style={'description_width': 'initial'})
    option_type_input = widgets.Dropdown(options=['call', 'put'], value='call', description='Option Type:', style={'description_width': 'initial'})
    
    run_button = widgets.Button(description='Calculate and Plot')
    output = widgets.Output()
    
    def on_button_click(b):
        with output:
            clear_output(wait=True)
            
            S = S_input.value
            K = K_input.value
            T = T_input.value
            r = r_input.value
            sigma = sigma_input.value
            num_simulations = num_simulations_input.value
            shown_simulations = shown_simulations_input.value
            num_steps = num_steps_input.value
            confidence_level = confidence_level_input.value
            option_type = option_type_input.value
            
            # Input validation
            if S <= 0:
                print("Error: Stock Price (S) must be positive.")
                return
            if K <= 0:
                print("Error: Strike Price (K) must be positive.")
                return
            if T <= 0:
                print("Error: Time to Maturity (T) must be positive.")
                return
            if sigma < 0:
                print("Error: Volatility (σ) must be non-negative.")
                return
            
            # Pricing
            bs_price = black_scholes(S, K, T, r, sigma, option_type)
            mc_price, paths, confidence_interval = monte_carlo(S, K, T, r, sigma, num_simulations, num_steps, confidence_level, option_type)
            
            print(f"Option Type: {option_type.capitalize()}")
            print(f"Black-Scholes Price: €{bs_price:.4f}")
            print(f"Monte Carlo Price: €{mc_price:.4f}")
            print(f"Difference: €{abs(bs_price - mc_price):.4f}")
            print(f"{confidence_level*100:.1f}% Confidence Interval: [€{mc_price - confidence_interval:.4f}, €{mc_price + confidence_interval:.4f}]")
            
            # Check put-call parity
            bs_call = black_scholes(S, K, T, r, sigma, 'call')
            bs_put = black_scholes(S, K, T, r, sigma, 'put')
            parity_check, lhs, rhs = check_put_call_parity(S, K, T, r, bs_call, bs_put)
            display(HTML("<br>Put-Call Parity Check:"))
            display(HTML(f"Left Hand Side (C + Ke<sup>-rT</sup>): €{lhs:.4f}"))
            display(HTML(f"Right Hand Side (P + S): €{rhs:.4f}"))
            display(HTML(f"<b>Parity {'holds' if parity_check else 'does not hold'}</b>"))




            #  ploting using plotly
            fig = make_subplots(rows=2, cols=1, subplot_titles=("Stock Price Paths", "Monte Carlo Convergence"), vertical_spacing=0.1)
            

            # Plot stock price paths
            for i in range(min(shown_simulations, num_simulations)):
                fig.add_trace(go.Scatter(y=paths[i], mode='lines', line=dict(width=1), showlegend=False), row=1, col=1)
            
            fig.update_xaxes(title_text="Time Steps", row=1, col=1)
            fig.update_yaxes(title_text="Stock Price (€)", row=1, col=1)
            
            # convergence of the monte carlo simulations
            if option_type == 'call':
                payoffs = np.maximum(paths[:, -1] - K, 0)
            else:
                payoffs = np.maximum(K - paths[:, -1], 0)
            option_prices = np.exp(-r * T) * payoffs
            convergence = np.cumsum(option_prices) / np.arange(1, num_simulations + 1)
            
            std_errors = np.std(option_prices, ddof=1) / np.sqrt(np.arange(1, num_simulations + 1))
            confidence_intervals = norm.ppf((1 + confidence_level) / 2) * std_errors
            
            # Plot convergence with confidence interval and add Black-Scholes price
            fig.add_trace(go.Scatter(y=convergence, mode='lines', name='Monte Carlo', line=dict(color='blue')), row=2, col=1)
            fig.add_trace(go.Scatter(y=convergence + confidence_intervals, mode='lines', line=dict(width=0), showlegend=False), row=2, col=1)
            fig.add_trace(go.Scatter(y=convergence - confidence_intervals, mode='lines', line=dict(width=0), 
                                     fill='tonexty', fillcolor='rgba(0,100,80,0.2)', name='Confidence Interval'), row=2, col=1)
            fig.add_trace(go.Scatter(y=[bs_price] * num_simulations, mode='lines', name='Black-Scholes (€)', line=dict(color='red')), row=2, col=1)
            
            fig.update_xaxes(title_text="Number of Simulations", row=2, col=1)
            fig.update_yaxes(title_text="Option Price (€)", row=2, col=1)
            
            fig.update_layout(height=1000, width=1300, title_text=f"Black-Scholes Monte Carlo Simulation ({option_type.capitalize()} Option)",
                              legend=dict(yanchor="top", y=0.45, xanchor="left", x=0.85))
            fig.show()
    
    run_button.on_click(on_button_click)
    
    display(widgets.VBox([
        widgets.HBox([S_input, K_input, T_input]),
        widgets.HBox([r_input, sigma_input, option_type_input]),
        widgets.HBox([num_simulations_input, shown_simulations_input, num_steps_input]),
        confidence_level_input,
        run_button
    ]))
    display(output)


In [6]:

black_scholes_monte_carlo_gui()


Output()